In [17]:
import glob 
import numpy as np
from keras.layers import LSTM,Dense,Embedding,Dropout
from keras.models import Sequential
from keras.utils import np_utils 
from music21 import converter,instrument,note,chord,stream
import pickle

In [18]:
def create_seq(notes,n_vocab,pitchname):
    length=50
    #mapping each node to int
    note_to_int=dict((note,number) for number,note in enumerate(pitchname))
    input_seq=[]
    output_seq=[]
    for i in range(0,len(notes)-length,1):
        in_seq=notes[i:i+length]
        out_seq=notes[i+length]
        input_seq.append([note_to_int[char] for char in in_seq])
        output_seq.append(note_to_int[out_seq])
    n_pattern=len(input_seq)
    
    #reshaping the input to amke it compatible with lstm
    input_seq_normalized=np.reshape(input_seq,(n_pattern,length,1))
    #normalizing
    input_seq_normalized=input_seq_normalized/float(n_vocab)
    return (input_seq,input_seq_normalized)

In [58]:
def load_model(input_seq,n_vocab):
    model=Sequential()
    model.add(LSTM(256,input_shape=(input_seq.shape[1],input_seq.shape[2]),return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512,return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(256))
    model.add(Dense(256,activation="relu"))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab,activation="softmax"))
    model.compile(loss="categorical_crossentropy",optimizer="rmsprop")
    model.load_weights("weights.hdf5")
    return model

In [69]:
def predict_using_model(input_seq,n_vocab,model,pitchname):
    # now we want our model to predict the music using given input seq 
    # we will use input sequence from file we save in training process hence to access the file randomly we use 
    #random variable available in numpy
    start=np.random.randint(1,len(input_seq)-1)
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern=input_seq[start]
    pred_seq=[]
    #we will generate 500 o/p which will give music of about 2 minutes
    for note_index in range(600):
        predicted_input=np.reshape(pattern,(1,len(pattern),1))
        predicted_input=predicted_input/float(n_vocab)   #normalization
        prediction=model.predict(predicted_input,verbose=0)
        index=np.argmax(prediction)
        result=int_to_note[index]
        pred_seq.append(result)
        
        pattern.append(index)
        pattern=pattern[1:len(pattern)]
    return pred_seq


In [70]:
def create_midi_file(pred_seq):
    offset=0
    output_seq=[]
    for pattern in pred_seq:
        if ("." in pattern) or pattern.isdigit():
            notes_in_chord=pattern.split(".")
            notes=[]
            for current_notes in notes_in_chord:
                new_note=note.Note(int(current_notes))
                new_note.storedInstrument=instrument.Piano()
                notes.append(new_note)
            new_chord=chord.Chord(notes)
            new_chord.offset=offset
            output_seq.append(new_chord)
        else:
            new_note=note.Note(pattern)
            new_note.offset=offset
            new_note.storedInstrument=instrument.Piano()
            output_seq.append(new_note)
        offset=offset+0.5
    midi_stream=stream.Stream(output_seq)
    midi_stream.write("midi",fp="output_music.mid")

# create_midi_file

Now that we have all the encoded representations of the notes and chords in an array we can start decoding them and creating an array of Note and Chord objects.

First we have to determine whether the output we are decoding is a Note or a Chord.

If the pattern is a Chord, we have to split the string up into an array of notes. Then we loop through the string representation of each note and create a Note object for each of them. Then we can create a Chord object containing each of these notes.


If the pattern is a Note, we create a Note object using the string representation of the pitch contained in the pattern.


In [71]:
with open("notes","rb") as filepath:
    notes=pickle.load(filepath)
pitchnames=sorted(set(pitch for pitch in notes))
n_vocab=len(set(notes))
network_input, normalized_input = create_seq(notes, n_vocab, pitchnames)
model = load_model(normalized_input, n_vocab)
prediction_output = predict_using_model(network_input,n_vocab,model,pitchnames)
create_midi_file(prediction_output)